### Data sources
- LEHD Origin-Destination Employment Statistics (LODES): The definition of variable codes, datasets, etc. can be found at the latest [LODES 7.3 Technical Documentation](https://lehd.ces.census.gov/data/lodes/LODES7/LODESTechDoc7.3.pdf). All LEHD Origin-Destination Employment Statistics (LODES) data are available, as described in the LODES documentation above. No changes have been made to the original CSV files. Data are available from 2002 to 2015. See the documentation above for caveats.
- Driving Times and Distances Dataset: Census tracts are 2010 vintage, and the columns are the origin tract, destination travel, travel time in minutes, and travel distance in miles. These data were calculated by the Data Science team at the Urban Institute. See [Github repo](https://github.com/UI-Research/spark-osrm).

In [1]:
import pyspark
from pyspark.sql.functions import *
from IPython.core.interactiveshell import InteractiveShell

warnings.filterwarnings(action='once')
InteractiveShell.ast_node_interactivity = "all"

In [2]:
spark = SparkSession.builder \
    .appName('pyspark-exploration') \
    .config('spark.driver.cores', '2') \
    .config('spark.executor.memory', '8gb') \
    .config('spark.executor.cores', '2') \
    .getOrCreate()     

In [3]:
def debug(df):
    """
    Function to pretty print the toDebugString
    """
    for rddstring in df.rdd.toDebugString().split('\n'):
        print rddstring.strip()

Load in data and see what it looks like

In [4]:
drive = spark.read.parquet('s3://lsdm-emr-util/lsdm-data/travel-times/drive_times.parquet')
od = spark.read.parquet('s3://lsdm-emr-util/lsdm-data/lodes/od/od.parquet')

In [5]:
print((drive.count(), len(drive.columns)))
drive.take(2)
drive.dtypes

(122004331, 4)


[Row(from_tract=u'36103146402', to_tract=u'42091207003', miles=141.2, minutes=184.1),
 Row(from_tract=u'36103146402', to_tract=u'42091209000', miles=162.7, minutes=203.4)]

[('from_tract', 'string'),
 ('to_tract', 'string'),
 ('miles', 'double'),
 ('minutes', 'double')]

In [6]:
print((od.count(), len(od.columns)))
od.take(2)
od.dtypes

(1577789908, 14)


[Row(w_geocode=u'271630714002025', h_geocode=u'271630712082020', s000=1, sa01=0, sa02=1, sa03=0, se01=0, se02=1, se03=0, si01=1, si02=0, si03=0, createdate=u'20160219', year=2012),
 Row(w_geocode=u'271630714002025', h_geocode=u'271630712083004', s000=1, sa01=0, sa02=1, sa03=0, se01=0, se02=1, se03=0, si01=1, si02=0, si03=0, createdate=u'20160219', year=2012)]

[('w_geocode', 'string'),
 ('h_geocode', 'string'),
 ('s000', 'int'),
 ('sa01', 'int'),
 ('sa02', 'int'),
 ('sa03', 'int'),
 ('se01', 'int'),
 ('se02', 'int'),
 ('se03', 'int'),
 ('si01', 'int'),
 ('si02', 'int'),
 ('si03', 'int'),
 ('createdate', 'string'),
 ('year', 'int')]

Make census tract columns in origin-destination data

In [7]:
od = od.withColumn('h_tract', substring(od.h_geocode, 0, 11))\
        .withColumn('w_tract', substring(od.w_geocode, 0, 11))

Check out lineage and partitions of the two dataframes

In [8]:
debug(od)

(137) MapPartitionsRDD[25] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[24] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[23] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   FileScanRDD[22] at javaToPython at NativeMethodAccessorImpl.java:0 []


In [9]:
debug(drive)

(33) MapPartitionsRDD[29] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[28] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[27] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   FileScanRDD[26] at javaToPython at NativeMethodAccessorImpl.java:0 []


In [10]:
od.rdd.getNumPartitions()
drive.rdd.getNumPartitions()

137

33

### Join origin-destination and driving dataframes

Repartition od before joining



In [11]:
od = od.repartition(8)

Left join join with driving, giving us travel times for each origin-destination pair.  
Assumption: travel time and distance for a census tract is the same for all its comprising block groups.

In [12]:
df = od.join(drive, [drive.from_tract == od.h_tract, drive.to_tract == od.w_tract])

Resulting dataframe is split across 200 partitions, as we can see from the getNumPartitions method

In [13]:
df.rdd.getNumPartitions()

200

In [14]:
debug(df)

(200) MapPartitionsRDD[45] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[44] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[43] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   ZippedPartitionsRDD2[42] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[36] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   ShuffledRowRDD[35] at javaToPython at NativeMethodAccessorImpl.java:0 []
+-(8) MapPartitionsRDD[34] at javaToPython at NativeMethodAccessorImpl.java:0 []
|  ShuffledRowRDD[33] at javaToPython at NativeMethodAccessorImpl.java:0 []
+-(137) MapPartitionsRDD[32] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[31] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   FileScanRDD[30] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   MapPartitionsRDD[41] at javaToPython at NativeMethodAccessorImpl.java:0 []
|   ShuffledRowRDD[40] at javaToPython at NativeMe

### Grouped aggregation - average travel time by year